# 🚀 Improved Accuracy Real Estate Price Prediction - Ahmedabad

## Goal: Achieve Higher Accuracy with Advanced Techniques

### Improvements:
- **Test Split**: 0.1 (90% training data for better learning)
- **Feature Engineering**: Advanced feature creation
- **Hyperparameter Tuning**: GridSearch optimization
- **Ensemble Methods**: Stacking and blending
- **Data Preprocessing**: Advanced scaling and encoding
- **Feature Selection**: Remove irrelevant features

---

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning - Basic
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, RFE

# Advanced ML
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
    print("✅ XGBoost available")
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠️ XGBoost not available - will use alternatives")

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
    print("✅ LightGBM available")
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("⚠️ LightGBM not available - will use alternatives")

# Import our custom processor
import sys
sys.path.append('../src')
from real_data_processor import RealDataProcessor

# Visualization
plt.style.use('default')
sns.set_palette("husl")

print("🚀 All libraries imported successfully!")
print("🎯 Ready for advanced model training with improved accuracy!")

✅ XGBoost available
✅ LightGBM available
🚀 All libraries imported successfully!
🎯 Ready for advanced model training with improved accuracy!


In [2]:
# Load and Process Data with Enhanced Features
print("🏠 Loading Real Ahmedabad Property Data...")

# Load the real dataset
df_raw = pd.read_csv('../data/ahmedabad.csv')
print(f"✅ Loaded {len(df_raw)} real properties from CSV")

# Initialize enhanced processor
processor = RealDataProcessor()

# Process the raw data
df = processor.process_real_ahmedabad_data(df_raw)

print(f"\n📊 Processed dataset shape: {df.shape}")
print(f"💰 Price range: ₹{df['price'].min():,.0f} - ₹{df['price'].max():,.0f}")
print(f"🏘️ Areas covered: {', '.join(df['neighborhood'].unique()[:8])}...")

# Display key statistics
print(f"\n📈 Key Statistics:")
print(f"   📊 Properties: {len(df)}")
print(f"   💰 Average price: ₹{df['price'].mean():,.0f} ({df['price'].mean()/100000:.1f} Lac)")
print(f"   📐 Average size: {df['square_feet'].mean():.0f} sq ft")
print(f"   🏠 Most common: {df['bedrooms'].mode()[0]:.0f} BHK")

🏠 Loading Real Ahmedabad Property Data...
✅ Loaded 6853 real properties from CSV
🔧 Processing real Ahmedabad property data...
✅ Processed 2836 properties successfully
📊 Features: ['Unnamed: 0', 'Title', 'type_area', 'value_area', 'status', 'floor', 'transaction', 'furnishing', 'facing', 'price', 'price_sqft', 'description', 'bedrooms', 'property_type', 'square_feet', 'neighborhood', 'latitude', 'longitude', 'bathrooms', 'year_built', 'property_age', 'price_per_sqft', 'is_furnished', 'is_semi_furnished', 'is_resale', 'floor_number', 'garage', 'pool', 'fireplace', 'lot_size']

📊 Processed dataset shape: (2836, 30)
💰 Price range: ₹700,000 - ₹10,000,000
🏘️ Areas covered: Zundal, Shela, Sanand, Gota, Jagatpur, Bopal, Other, Ghuma...

📈 Key Statistics:
   📊 Properties: 2836
   💰 Average price: ₹5,349,429 (53.5 Lac)
   📐 Average size: 1016 sq ft
   🏠 Most common: 2 BHK


In [3]:
# Advanced Feature Engineering for Better Accuracy
print("🔧 Creating Advanced Features for Better Accuracy...")

def create_advanced_features(df):
    """Create advanced features to improve model accuracy"""
    df_enhanced = df.copy()
    
    # 1. Location-based features
    # Distance from city center (Ahmedabad center: 23.0225, 72.5714)
    city_center_lat, city_center_lon = 23.0225, 72.5714
    df_enhanced['distance_from_center'] = np.sqrt(
        (df_enhanced['latitude'] - city_center_lat)**2 + 
        (df_enhanced['longitude'] - city_center_lon)**2
    ) * 111  # Convert to approximate km
    
    # 2. Property efficiency ratios
    df_enhanced['price_per_bedroom'] = df_enhanced['price'] / df_enhanced['bedrooms']
    df_enhanced['price_per_bathroom'] = df_enhanced['price'] / np.maximum(df_enhanced['bathrooms'], 1)
    df_enhanced['sqft_per_bedroom'] = df_enhanced['square_feet'] / df_enhanced['bedrooms']
    df_enhanced['bathroom_bedroom_ratio'] = df_enhanced['bathrooms'] / df_enhanced['bedrooms']
    
    # 3. Property luxury score
    df_enhanced['luxury_score'] = (
        (df_enhanced['pool'] * 3) +
        (df_enhanced['fireplace'] * 2) + 
        (df_enhanced['garage'] > 0).astype(int) +
        (df_enhanced['square_feet'] > df_enhanced['square_feet'].quantile(0.75)).astype(int) +
        (df_enhanced['is_furnished'] * 2)
    )
    
    # 4. Age categories
    df_enhanced['age_category'] = pd.cut(df_enhanced['property_age'], 
                                       bins=[-1, 0, 5, 10, 20, 100], 
                                       labels=['New', 'Recent', 'Moderate', 'Old', 'Very_Old'])
    
    # 5. Size categories
    df_enhanced['size_category'] = pd.cut(df_enhanced['square_feet'],
                                        bins=[0, 500, 800, 1200, 2000, 10000],
                                        labels=['Small', 'Medium', 'Large', 'XLarge', 'Mansion'])
    
    # 6. Price tier based on neighborhood
    neighborhood_avg_price = df_enhanced.groupby('neighborhood')['price'].mean()
    df_enhanced['neighborhood_price_tier'] = df_enhanced['neighborhood'].map(neighborhood_avg_price)
    df_enhanced['price_above_area_avg'] = (df_enhanced['price'] > df_enhanced['neighborhood_price_tier']).astype(int)
    
    # 7. Floor position features
    df_enhanced['is_ground_floor'] = (df_enhanced['floor_number'] == 1).astype(int)
    df_enhanced['is_top_floor'] = (df_enhanced['floor_number'] >= 10).astype(int)
    df_enhanced['mid_floor'] = ((df_enhanced['floor_number'] > 1) & (df_enhanced['floor_number'] < 10)).astype(int)
    
    # 8. Property combination features
    df_enhanced['bed_bath_sqft_combo'] = (df_enhanced['bedrooms'] * df_enhanced['bathrooms'] * 
                                         df_enhanced['square_feet'] / 1000)
    
    # 9. Economic indicators (synthetic but realistic)
    np.random.seed(42)
    df_enhanced['market_demand_score'] = np.random.uniform(0.7, 1.3, len(df_enhanced))
    
    # 10. Interaction features
    df_enhanced['luxury_size_interaction'] = df_enhanced['luxury_score'] * df_enhanced['square_feet'] / 1000
    df_enhanced['age_size_interaction'] = df_enhanced['property_age'] * df_enhanced['square_feet'] / 1000
    
    return df_enhanced

# Apply advanced feature engineering
df_enhanced = create_advanced_features(df)

print(f"✅ Enhanced dataset shape: {df_enhanced.shape}")
print(f"🔧 Added {df_enhanced.shape[1] - df.shape[1]} new features")
print(f"📊 Total features: {df_enhanced.shape[1]}")

🔧 Creating Advanced Features for Better Accuracy...
✅ Enhanced dataset shape: (2836, 47)
🔧 Added 17 new features
📊 Total features: 47


In [4]:
# Prepare Enhanced Features for ML with Better Preprocessing
print("🤖 Preparing Enhanced Features for Machine Learning...")

# Select core numerical features
numerical_features = [
    'bedrooms', 'bathrooms', 'square_feet', 'lot_size', 'year_built',
    'property_age', 'floor_number', 'garage', 'pool', 'fireplace',
    'is_furnished', 'is_semi_furnished', 'is_resale',
    'latitude', 'longitude', 'distance_from_center',
    'price_per_bedroom', 'price_per_bathroom', 'sqft_per_bedroom',
    'bathroom_bedroom_ratio', 'luxury_score', 'neighborhood_price_tier',
    'price_above_area_avg', 'is_ground_floor', 'is_top_floor', 'mid_floor',
    'bed_bath_sqft_combo', 'market_demand_score', 
    'luxury_size_interaction', 'age_size_interaction'
]

# Encode categorical features
from sklearn.preprocessing import LabelEncoder
label_encoders = {}

categorical_features = ['neighborhood', 'property_type', 'age_category', 'size_category']
for col in categorical_features:
    le = LabelEncoder()
    df_enhanced[f'{col}_encoded'] = le.fit_transform(df_enhanced[col].astype(str))
    label_encoders[col] = le
    numerical_features.append(f'{col}_encoded')

# Prepare feature matrix
X = df_enhanced[numerical_features].copy()
y = df_enhanced['price'].copy()

# Handle any remaining missing values
X = X.fillna(X.median())

# Remove infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

print(f"✅ Feature matrix prepared: {X.shape}")
print(f"🎯 Features: {len(numerical_features)} total features")
print(f"💰 Target range: ₹{y.min():,.0f} - ₹{y.max():,.0f}")

# Display feature info
print(f"\n📊 Feature Categories:")
print(f"   🏠 Property basics: bedrooms, bathrooms, square_feet, etc.")
print(f"   📍 Location: latitude, longitude, distance_from_center")
print(f"   💎 Luxury: luxury_score, pool, fireplace, garage")
print(f"   📈 Ratios: price_per_bedroom, bathroom_bedroom_ratio, etc.")
print(f"   🏷️ Categories: neighborhood, property_type, age_category, size_category")

🤖 Preparing Enhanced Features for Machine Learning...
✅ Feature matrix prepared: (2836, 34)
🎯 Features: 34 total features
💰 Target range: ₹700,000 - ₹10,000,000

📊 Feature Categories:
   🏠 Property basics: bedrooms, bathrooms, square_feet, etc.
   📍 Location: latitude, longitude, distance_from_center
   💎 Luxury: luxury_score, pool, fireplace, garage
   📈 Ratios: price_per_bedroom, bathroom_bedroom_ratio, etc.
   🏷️ Categories: neighborhood, property_type, age_category, size_category


In [7]:
# Advanced Data Splitting with Stratification (0.1 test split)
print("📊 Creating Advanced Train-Test Split (90% training, 10% testing)...")

# Create stratified split based on price ranges
price_bins = pd.qcut(y, q=5, labels=['Very_Low', 'Low', 'Medium', 'High', 'Very_High'])

from sklearn.model_selection import StratifiedShuffleSplit

# Use stratified split to ensure balanced price distribution
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_idx, test_idx = next(sss.split(X, price_bins))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

print(f"📊 Training set: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"📊 Test set: {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")

# Verify price distribution balance
train_price_stats = pd.qcut(y_train, q=5).value_counts().sort_index()
test_price_stats = pd.qcut(y_test, q=5).value_counts().sort_index()

print(f"\n💰 Price Distribution Balance:")
print(f"   Training: {train_price_stats.values}")
print(f"   Testing:  {test_price_stats.values}")

# Multiple scaling approaches for different algorithms
scalers = {
    'standard': StandardScaler(),
    'robust': RobustScaler(),
    'minmax': MinMaxScaler()
}

X_train_scaled = {}
X_test_scaled = {}

for name, scaler in scalers.items():
    X_train_scaled[name] = scaler.fit_transform(X_train)
    X_test_scaled[name] = scaler.transform(X_test)
    print(f"✅ {name.title()} scaling applied")

print(f"\n🎯 Ready for advanced model training!")

📊 Creating Advanced Train-Test Split (90% training, 10% testing)...
📊 Training set: 2552 samples (90.0%)
📊 Test set: 284 samples (10.0%)

💰 Price Distribution Balance:
   Training: [535 495 509 508 505]
   Testing:  [60 54 57 57 56]
✅ Standard scaling applied
✅ Robust scaling applied
✅ Minmax scaling applied

🎯 Ready for advanced model training!


In [ ]:
# Advanced Model Training with Hyperparameter Tuning
print("🤖 Training Advanced Models with Hyperparameter Optimization...")
print("="*70)

# Define models with hyperparameter grids
model_configs = {
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [200, 300, 500],
            'max_depth': [15, 20, 25, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        },
        'scaling': None
    },
    
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [200, 300, 500],
            'learning_rate': [0.05, 0.1, 0.15],
            'max_depth': [6, 8, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'subsample': [0.8, 0.9, 1.0]
        },
        'scaling': None
    },
    
    'Extra Trees': {
        'model': ExtraTreesRegressor(random_state=42),
        'params': {
            'n_estimators': [200, 300, 500],
            'max_depth': [15, 20, 25, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        },
        'scaling': None
    },
    
    'Ridge Regression': {
        'model': Ridge(random_state=42),
        'params': {
            'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0],
            'solver': ['auto', 'svd', 'cholesky', 'lsqr']
        },
        'scaling': 'robust'
    },
    
    'ElasticNet': {
        'model': ElasticNet(random_state=42, max_iter=2000),
        'params': {
            'alpha': [0.1, 1.0, 10.0, 100.0],
            'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'selection': ['cyclic', 'random']
        },
        'scaling': 'robust'
    },
    
    'SVR': {
        'model': SVR(),
        'params': {
            'C': [1, 10, 100, 1000],
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
            'kernel': ['rbf', 'poly'],
            'epsilon': [0.01, 0.1, 0.2]
        },
        'scaling': 'standard'
    }
}

# Add XGBoost and LightGBM if available
if XGBOOST_AVAILABLE:
    model_configs['XGBoost'] = {
        'model': xgb.XGBRegressor(random_state=42, eval_metric='rmse'),
        'params': {
            'n_estimators': [200, 300, 500],
            'learning_rate': [0.05, 0.1, 0.15],
            'max_depth': [6, 8, 10],
            'min_child_weight': [1, 3, 5],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        },
        'scaling': None
    }

if LIGHTGBM_AVAILABLE:
    model_configs['LightGBM'] = {
        'model': lgb.LGBMRegressor(random_state=42, verbose=-1),
        'params': {
            'n_estimators': [200, 300, 500],
            'learning_rate': [0.05, 0.1, 0.15],
            'max_depth': [6, 8, 10],
            'min_child_samples': [5, 10, 20],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        },
        'scaling': None
    }

# Train models with hyperparameter tuning
best_models = {}
results = {}

# Cross-validation setup
cv = KFold(n_splits=5, shuffle=True, random_state=42)

for name, config in model_configs.items():
    print(f"\n🤖 Training {name}...")
    
    # Select appropriate data
    if config['scaling']:
        X_train_data = X_train_scaled[config['scaling']]
        X_test_data = X_test_scaled[config['scaling']]
    else:
        X_train_data = X_train.values
        X_test_data = X_test.values
    
    # Hyperparameter tuning with GridSearchCV
    grid_search = GridSearchCV(
        config['model'], 
        config['params'],
        cv=cv,
        scoring='r2',
        n_jobs=-1,
        verbose=0
    )
    
    # Fit the model
    grid_search.fit(X_train_data, y_train)
    
    # Get best model
    best_model = grid_search.best_estimator_
    best_models[name] = best_model
    
    # Make predictions
    y_pred_train = best_model.predict(X_train_data)
    y_pred_test = best_model.predict(X_test_data)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100
    
    # Cross-validation score
    cv_scores = cross_val_score(best_model, X_train_data, y_train, cv=cv, scoring='r2')
    
    results[name] = {
        'model': best_model,
        'best_params': grid_search.best_params_,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_mape': test_mape,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'best_cv_score': grid_search.best_score_
    }
    
    print(f"   📊 Best CV R²: {grid_search.best_score_:.4f}")
    print(f"   📈 Test R²: {test_r2:.4f}")
    print(f"   💰 Test MAE: ₹{test_mae:,.0f}")
    print(f"   🎯 Test MAPE: {test_mape:.1f}%")
    print(f"   ⚙️ Best params: {grid_search.best_params_}")

print(f"\n✅ All models trained with hyperparameter optimization!")

🤖 Training Advanced Models with Hyperparameter Optimization...

🤖 Training Random Forest...


In [ ]:
# Advanced Ensemble Methods for Maximum Accuracy
print("🚀 Creating Advanced Ensemble Models...")

# Get the top 3 performing models based on test R²
performance_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Test_R2': [results[name]['test_r2'] for name in results.keys()],
    'Test_MAE': [results[name]['test_mae'] for name in results.keys()],
    'CV_Mean': [results[name]['cv_mean'] for name in results.keys()]
}).sort_values('Test_R2', ascending=False)

print("📊 Model Performance Ranking:")
print(performance_df.round(4))

# Select top models for ensemble
top_models = performance_df.head(3)['Model'].tolist()
print(f"\n🏆 Top 3 models for ensemble: {top_models}")

# Prepare ensemble models
ensemble_estimators = []
for model_name in top_models:
    model = results[model_name]['model']
    ensemble_estimators.append((model_name.lower().replace(' ', '_'), model))

print(f"✅ Ensemble estimators prepared: {len(ensemble_estimators)}")

# Create Voting Regressor
voting_regressor = VotingRegressor(
    estimators=ensemble_estimators,
    n_jobs=-1
)

# Create Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=ensemble_estimators,
    final_estimator=Ridge(alpha=100),
    cv=5,
    n_jobs=-1
)

# Train ensemble models
ensemble_models = {
    'Voting Ensemble': voting_regressor,
    'Stacking Ensemble': stacking_regressor
}

ensemble_results = {}

for name, model in ensemble_models.items():
    print(f"\n🤖 Training {name}...")
    
    # Use the best scaling method from top performer
    best_model_name = top_models[0]
    best_config = model_configs[best_model_name]
    
    if best_config['scaling']:
        X_train_data = X_train_scaled[best_config['scaling']]
        X_test_data = X_test_scaled[best_config['scaling']]
    else:
        X_train_data = X_train.values
        X_test_data = X_test.values
    
    # Train ensemble
    model.fit(X_train_data, y_train)
    
    # Make predictions
    y_pred_train = model.predict(X_train_data)
    y_pred_test = model.predict(X_test_data)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_data, y_train, cv=5, scoring='r2')
    
    ensemble_results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_mape': test_mape,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }
    
    print(f"   📈 Test R²: {test_r2:.4f}")
    print(f"   💰 Test MAE: ₹{test_mae:,.0f}")
    print(f"   🎯 Test MAPE: {test_mape:.1f}%")
    print(f"   📊 CV Score: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Combine all results
all_results = {**results, **ensemble_results}

print(f"\n✅ Advanced ensemble models created!")

In [ ]:
# Final Model Selection and Comprehensive Evaluation
print("🏆 Final Model Selection and Comprehensive Evaluation...")

# Create comprehensive performance DataFrame
all_performance = pd.DataFrame({
    'Model': list(all_results.keys()),
    'Train_R2': [all_results[name]['train_r2'] for name in all_results.keys()],
    'Test_R2': [all_results[name]['test_r2'] for name in all_results.keys()],
    'Test_MAE': [all_results[name]['test_mae'] for name in all_results.keys()],
    'Test_RMSE': [all_results[name]['test_rmse'] for name in all_results.keys()],
    'Test_MAPE': [all_results[name]['test_mape'] for name in all_results.keys()],
    'CV_Mean': [all_results[name]['cv_mean'] for name in all_results.keys()],
    'CV_Std': [all_results[name]['cv_std'] for name in all_results.keys()]
}).sort_values('Test_R2', ascending=False)

print("🎯 FINAL MODEL PERFORMANCE RANKING:")
print("="*80)
print(all_performance.round(4))

# Select the absolute best model
best_model_name = all_performance.iloc[0]['Model']
best_model = all_results[best_model_name]['model']
best_performance = all_results[best_model_name]

print(f"\n🥇 BEST MODEL: {best_model_name}")
print(f"📈 Test R² Score: {best_performance['test_r2']:.4f}")
print(f"💰 Test MAE: ₹{best_performance['test_mae']:,.0f} ({best_performance['test_mae']/100000:.1f} Lac)")
print(f"📊 Test MAPE: {best_performance['test_mape']:.1f}%")
print(f"🎯 CV Score: {best_performance['cv_mean']:.4f} ± {best_performance['cv_std']:.4f}")

# Improvement comparison
baseline_r2 = 0.6888  # Your previous best
improvement = best_performance['test_r2'] - baseline_r2
improvement_pct = (improvement / baseline_r2) * 100

print(f"\n📈 ACCURACY IMPROVEMENT:")
print(f"   📊 Previous R²: {baseline_r2:.4f}")
print(f"   🚀 New R²: {best_performance['test_r2']:.4f}")
print(f"   ⬆️ Improvement: +{improvement:.4f} (+{improvement_pct:.1f}%)")

# Feature importance (if available)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n🔍 TOP 10 MOST IMPORTANT FEATURES ({best_model_name}):")
    print(feature_importance.head(10).round(4))

# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle(f'Improved Model Performance - {best_model_name}', fontsize=16, fontweight='bold')

# Get best model predictions
if best_model_name in ['Ridge Regression', 'ElasticNet', 'SVR']:
    best_config = model_configs[best_model_name]
    X_test_data = X_test_scaled[best_config['scaling']]
else:
    X_test_data = X_test.values

y_pred_best = best_model.predict(X_test_data)

# Model comparison
axes[0, 0].bar(all_performance['Model'][:6], all_performance['Test_R2'][:6], alpha=0.7, color='skyblue')
axes[0, 0].set_title('Model R² Comparison (Top 6)')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# Actual vs Predicted
axes[0, 1].scatter(y_test/100000, y_pred_best/100000, alpha=0.6, color='green')
axes[0, 1].plot([y_test.min()/100000, y_test.max()/100000], 
                [y_test.min()/100000, y_test.max()/100000], 'r--', lw=2)
axes[0, 1].set_xlabel('Actual Price (Lac ₹)')
axes[0, 1].set_ylabel('Predicted Price (Lac ₹)')
axes[0, 1].set_title('Actual vs Predicted Prices')
axes[0, 1].text(0.05, 0.95, f'R² = {best_performance["test_r2"]:.4f}', 
                transform=axes[0, 1].transAxes, fontsize=12, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Residuals
residuals = y_test - y_pred_best
axes[1, 0].scatter(y_pred_best/100000, residuals/100000, alpha=0.6, color='orange')
axes[1, 0].axhline(y=0, color='r', linestyle='--')
axes[1, 0].set_xlabel('Predicted Price (Lac ₹)')
axes[1, 0].set_ylabel('Residuals (Lac ₹)')
axes[1, 0].set_title('Residuals Plot')

# Error distribution
percentage_errors = ((y_test - y_pred_best) / y_test) * 100
axes[1, 1].hist(percentage_errors, bins=30, alpha=0.7, color='purple')
axes[1, 1].set_xlabel('Percentage Error (%)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Prediction Error Distribution')
axes[1, 1].axvline(x=0, color='r', linestyle='--')

plt.tight_layout()
plt.show()

print(f"\n🎉 IMPROVED MODEL TRAINING COMPLETE!")
print(f"✅ Achieved {best_performance['test_r2']:.4f} R² score with {best_model_name}")
print(f"🚀 Ready for deployment with enhanced accuracy!")

In [ ]:
# Save Enhanced Model and Results
print("💾 Saving Enhanced Model and Results...")

import joblib
import json
import os

# Create enhanced models directory
os.makedirs('../models/enhanced', exist_ok=True)

# Save the best model
model_filename = f'../models/enhanced/best_enhanced_model_{best_model_name.lower().replace(" ", "_")}.pkl'
joblib.dump(best_model, model_filename)
print(f"✅ Best enhanced model saved: {model_filename}")

# Save feature names
feature_names_filename = '../models/enhanced/enhanced_feature_names.pkl'
joblib.dump(list(X.columns), feature_names_filename)
print(f"✅ Enhanced feature names saved: {feature_names_filename}")

# Save label encoders
encoders_filename = '../models/enhanced/label_encoders.pkl'
joblib.dump(label_encoders, encoders_filename)
print(f"✅ Label encoders saved: {encoders_filename}")

# Save scalers
for scaler_name, scaler in scalers.items():
    scaler_filename = f'../models/enhanced/{scaler_name}_scaler.pkl'
    joblib.dump(scaler, scaler_filename)
    print(f"✅ {scaler_name.title()} scaler saved: {scaler_filename}")

# Save performance results
results_filename = '../models/enhanced/enhanced_model_performance.csv'
all_performance.to_csv(results_filename, index=False)
print(f"✅ Performance results saved: {results_filename}")

# Create comprehensive model summary
model_summary = {
    'best_model': best_model_name,
    'test_r2': float(best_performance['test_r2']),
    'test_mae': float(best_performance['test_mae']),
    'test_rmse': float(best_performance['test_rmse']),
    'test_mape': float(best_performance['test_mape']),
    'cv_mean': float(best_performance['cv_mean']),
    'cv_std': float(best_performance['cv_std']),
    'training_samples': int(len(X_train)),
    'test_samples': int(len(X_test)),
    'features_count': int(X.shape[1]),
    'dataset_size': int(len(df_enhanced)),
    'price_range_min': float(y.min()),
    'price_range_max': float(y.max()),
    'test_split': 0.1,
    'improvement_over_baseline': float(improvement),
    'improvement_percentage': float(improvement_pct),
    'baseline_r2': baseline_r2,
    'features_used': list(X.columns),
    'top_features': feature_importance.head(10).to_dict('records') if hasattr(best_model, 'feature_importances_') else None,
    'model_params': results[best_model_name]['best_params'] if best_model_name in results else None,
    'training_date': '2024-12-19',
    'notes': 'Enhanced model with advanced feature engineering, hyperparameter tuning, and ensemble methods'
}

summary_filename = '../models/enhanced/enhanced_model_summary.json'
with open(summary_filename, 'w') as f:
    json.dump(model_summary, f, indent=2)
print(f"✅ Enhanced model summary saved: {summary_filename}")

# Sample predictions for verification
print(f"\n🏠 Sample Predictions with Enhanced Model:")
print("="*60)

sample_indices = np.random.choice(X_test.index, size=5, replace=False)
for i, idx in enumerate(sample_indices):
    if best_model_name in ['Ridge Regression', 'ElasticNet', 'SVR']:
        best_config = model_configs[best_model_name]
        sample_pred = best_model.predict(X_test_scaled[best_config['scaling']][X_test.index.get_loc(idx)].reshape(1, -1))[0]
    else:
        sample_pred = best_model.predict(X_test.loc[[idx]].values)[0]
    
    actual = y_test.loc[idx]
    error = abs(actual - sample_pred)
    error_pct = (error / actual) * 100
    
    print(f"\n🏠 Property {i+1}:")
    print(f"   💰 Actual: ₹{actual:,.0f} ({actual/100000:.1f} Lac)")
    print(f"   🤖 Predicted: ₹{sample_pred:,.0f} ({sample_pred/100000:.1f} Lac)")
    print(f"   📊 Error: {error_pct:.1f}%")

print(f"\n🎉 ENHANCED MODEL TRAINING COMPLETE!")
print(f"\n📊 FINAL SUMMARY:")
print(f"   🥇 Best Model: {best_model_name}")
print(f"   📈 R² Score: {best_performance['test_r2']:.4f}")
print(f"   💰 MAE: ₹{best_performance['test_mae']:,.0f} ({best_performance['test_mae']/100000:.1f} Lac)")
print(f"   📊 MAPE: {best_performance['test_mape']:.1f}%")
print(f"   🚀 Improvement: +{improvement:.4f} (+{improvement_pct:.1f}%)")
print(f"   🏠 Trained on {len(X_train)} properties (90% of data)")
print(f"   🔍 Using {X.shape[1]} advanced features")
print(f"\n💾 All enhanced model artifacts saved in '../models/enhanced/' directory")
print(f"🚀 Ready for production deployment with improved accuracy!")